In [9]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
import cv2
from ipywidgets import interact, IntRangeSlider, fixed
from scipy.optimize import curve_fit

In [11]:
path_name = "fotos_experimento"
path_list = os.listdir(path_name)

In [12]:
def ordenar(item):
    partes = item.split()
    num_principal = int(partes[0])
    num_placa = int(partes[2])
    hora = partes[3].replace('.jpg', '')
    return (num_principal, num_placa, hora)

path_list.sort(key=ordenar)


In [13]:
path_image_list = [f'{path_name}/' + i for i in path_list]

In [14]:
df = pd.DataFrame()
datas = []
samples = []

for path in path_image_list:
    for i in range(1,4):
        id = "placa " + str(i)
        if id in path:
            id_name = id
    
    data = path[-12:].split('.')[0].replace("_",":")

    datas.append(data)
    samples.append(id_name)


df['Hora'] = datas
df['Amostra'] = samples
df['Pasta'] = path_image_list

In [28]:



def detect_circle(path: str, max_min_radius: tuple,two_filters: tuple, plot_image=False, save_name=None):

    try:
        original_pil = Image.open(path)
        gray_pil = original_pil.convert("L")
        gray = np.array(gray_pil)

        gray_blur = cv2.GaussianBlur(gray, (9, 9), 2)

        circles = cv2.HoughCircles(
            gray_blur,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=100,
            param1=100,
            param2=30,
            minRadius=max_min_radius[0],
            maxRadius=max_min_radius[1]
        )

        frame = np.array(Image.open(path)) 
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            circles = sorted(circles, key=lambda x: x[2], reverse=True)

            x, y, r = circles[0]
            cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
            n = 18
            r_reduced = max(1, r - n)
            cv2.circle(frame, (x, y), r_reduced, (255, 0, 255), 2)

            mask = np.zeros_like(gray)
            cv2.circle(mask, (x, y), r_reduced, (255, 255, 255), -1)
            masked = cv2.bitwise_and(gray, mask)


        color_mask = cv2.bitwise_and(frame, frame, mask=mask)


        return color_mask,masked
    

    except:
        return None,None


def rgb_filter(image, 
               r_range=(0,255), 
               g_range=(0,255), 
               b_range=(0,255), 
               gray_range=(0,255),
               mode="rgb", 
               show_plot=True):
    
    if mode == "gray":
        mask = (image >= gray_range[0]) & (image <= gray_range[1])
        result = np.zeros_like(image)
        result[mask] = 255

    elif mode == "rgb":
        mask = (
            (image[:,:,0] >= r_range[0]) & (image[:,:,0] <= r_range[1]) &
            (image[:,:,1] >= g_range[0]) & (image[:,:,1] <= g_range[1]) &
            (image[:,:,2] >= b_range[0]) & (image[:,:,2] <= b_range[1])
        )
        result = np.zeros_like(image)
        result[mask] = [255, 255, 255]
    else:
        raise ValueError("mode deve ser 'rgb' ou 'gray'")

    if show_plot:
        plt.figure(figsize=(12,6))
        plt.subplot(1,2,1)
        plt.title("Original")
        cmap = "gray" if mode=="gray" else None
        plt.imshow(image, cmap=cmap)
        plt.axis("off")

        plt.subplot(1,2,2)
        plt.title("Filtrado")
        cmap = "gray" if mode=="gray" else None
        plt.imshow(result, cmap=cmap)
        plt.axis("off")
        plt.show()
    else:
        return result
    
def verhulst(t, K, A, r):
    return K / (1 + A * np.exp(-r * t))

def preprocess(df):
    all_data = []
    for j in range(1,3):
        filtro = df['Amostra'] == f'placa {j}'
        df_p_uso = df[filtro]

        dados_placa = []
        for pasta in df_p_uso['Pasta']:
            masked_rgb, masked = detect_circle(pasta,(200,210),(0,1))
            dados_placa.append((masked_rgb, masked))
        all_data.append(dados_placa)
    return all_data

def plot_with_params(all_data, 
                     r_range=(0,255), 
                     g_range=(0,255), 
                     b_range=(0,255), 
                     gray_range=(0,255),
                     mode="rgb"):
    matrix_medias = []
    for dados_placa in all_data:
        medias = []
        for masked_rgb, masked in dados_placa:
            # aplica filtro
            filtrado = rgb_filter(masked if mode=="gray" else masked_rgb,
                                  r_range, g_range, b_range, gray_range,
                                  mode=mode, show_plot=False)
            # calcula métrica
            media = np.count_nonzero(filtrado) / np.count_nonzero(masked)
            medias.append(media)

        # === NORMALIZAÇÃO PELA PRIMEIRA IMAGEM ===
        if len(medias) > 0:
            medias = np.array(medias)
            medias = medias - medias[0]  # desloca para que o primeiro ponto seja 0

        matrix_medias.append(medias)

    # === Ajusta modelo de Verhulst para cada placa ===
    verhulst_curvas = []
    parametros_fit = []
    for i in range(len(matrix_medias)):
        t = np.arange(len(matrix_medias[i]))
        y = np.array(matrix_medias[i])
        if np.any(y):  # só ajusta se tiver dados > 0
            p0 = [max(y), 1, 0.1]
            try:
                params, _ = curve_fit(verhulst, t, y, p0=p0, maxfev=10000)
            except:
                params = [np.nan, np.nan, np.nan]
        else:
            params = [np.nan, np.nan, np.nan]
        parametros_fit.append(params)

    # === Plota dados + curvas ajustadas ===
    plt.figure(figsize=(10,6))
    plt.grid(zorder=1)

    for i in range(len(matrix_medias)):
        tempo = np.arange(len(matrix_medias[i]))/2
        

        K, A, r = parametros_fit[i]
        if not np.isnan(K):
            t_fit = np.arange(len(matrix_medias[i]))/2
            Vr = np.array(verhulst(np.arange(len(matrix_medias[i])), K, A, r))*100
            plt.plot(t_fit, Vr, linewidth=5,linestyle="--", label=f"Fit Verhulst Placa {i+1}",zorder=5)
        
        plt.scatter(tempo, np.array(matrix_medias[i])*100, label=f'Placa {i+1}', s=30, edgecolors='black',zorder=10)

    plt.legend(fontsize=16)
    plt.ylabel("Estimativa de área coberta por leveduras(%)",fontsize=15)
    plt.xlabel("Tempo (h)",fontsize=16)
    plt.show()


# === Interface interativa ===
def interactive_verhulst(df, mode="rgb"):
    all_data = preprocess(df)  # roda só 1x
    
    if mode == "gray":
        interact(
            plot_with_params,
            all_data=fixed(all_data),
            r_range=fixed((0,255)),
            g_range=fixed((0,255)),
            b_range=fixed((0,255)),
            gray_range=IntRangeSlider(value=[100,200], min=0, max=255, description="Gray"),
            mode=fixed("gray")
        )
    else:  # rgb
        interact(
            plot_with_params,
            all_data=fixed(all_data),
            r_range=IntRangeSlider(value=[90,170], min=0, max=255, description="R"),
            g_range=IntRangeSlider(value=[64,183], min=0, max=255, description="G"),
            b_range=IntRangeSlider(value=[55,255], min=0, max=255, description="B"),
            gray_range=fixed((0,255)),
            mode=fixed("rgb")
        )


In [ ]:
interactive_verhulst(df, mode="gray")  

interactive(children=(IntRangeSlider(value=(100, 200), description='Gray', max=255), Output()), _dom_classes=(…

In [17]:
import imageio.v2 as imageio
import os

def criar_gif(df, placa="placa 1", saida="animacao.gif", duracao=0.3):
    """
    Cria um GIF com as imagens de uma placa.
    
    df       -> DataFrame que contém a coluna 'Amostra' e 'Pasta' (caminho das imagens)
    placa    -> Qual amostra/placa usar (ex: 'placa 1')
    saida    -> Nome do arquivo de saída (.gif)
    duracao  -> Tempo (s) entre frames
    """
    # Filtra os caminhos da placa escolhida
    caminhos = df[df['Amostra'] == placa]['Pasta'].tolist()

    # Lê todas as imagens
    frames = []
    for caminho in caminhos:
        if os.path.exists(caminho):
            img = imageio.imread(caminho)
            frames.append(img)

    # Cria o GIF
    if frames:
        imageio.mimsave(saida, frames, duration=duracao)
        print(f"✅ GIF salvo em: {saida} ({len(frames)} frames)")
    else:
        print("⚠️ Nenhuma imagem encontrada!")

# Exemplo de uso:


In [18]:
#criar_gif(df, placa="placa 2", saida="placa2.gif", duracao=0.01)
